In [9]:
import random
import csv
import pandas as pd
import numpy as np
import math

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
#%cd /content/drive/MyDrive/Colab Notebooks/AI_project

In [22]:
filename = "Vehicle_Routing_dataset.csv"
customer_data = pd.read_csv(filename, delimiter=';',usecols=['Node', 'Demand', 'x-coordinate', 'y-coordinate'])


# Preload customer data into lists and calculate distance matrix
num_customers = len(customer_data)
customer_demands = customer_data['Demand'].tolist()
customer_coordinates = [(row['x-coordinate'], row['y-coordinate']) for _, row in customer_data.iterrows()]
distance_matrix = np.zeros((num_customers, num_customers))

for i in range(num_customers):
    for j in range(i + 1, num_customers):
        distance = math.sqrt((customer_coordinates[i][0] - customer_coordinates[j][0])**2 +
                             (customer_coordinates[i][1] - customer_coordinates[j][1])**2)
        distance_matrix[i][j] = distance
        distance_matrix[j][i] = distance


def distance_between_customers(c1, c2):
    """Get the precomputed distance between two customers."""
    return distance_matrix[c1][c2]

In [23]:

def route_distance(route, customer_data):
    """Calculate the total distance of a route."""
    return sum(distance_between_customers(customer_data.iloc[route[i]][0], customer_data.iloc[route[i + 1]][0]) for i in range(len(route) - 1))


def total_distance_of_routes(routes, distance_matrix):
    """Calculate the total distance of routes, starting and ending at depot."""
    total_distance = 0
    for route in routes:
        # Including distance from depot to the first customer
        route_distance_with_depot = distance_matrix[0][route[0]]
        route_distance_with_depot += sum(distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))
        # Including distance from the last customer to the depot
        route_distance_with_depot += distance_matrix[route[-1]][0]
        total_distance += route_distance_with_depot
    return total_distance





In [24]:
def generate_individual(distance_matrix, vehicle_capacity, num_vehicles):
    individual = []
    customers = list(range(1, len(distance_matrix)))
    random.shuffle(customers)
    for _ in range(num_vehicles):
        route = []
        route_demand = 0
        i = 0
        while i < len(customers) and len(customers) != 0:
            c = customers[i]
            demand = customer_demands[c]
            if route_demand + demand <= vehicle_capacity:
                route.append(c)
                route_demand += demand
                customers.pop(i)  # Remove the customer from the list
            else:
                i += 1
        individual.append(route)
    return individual

def generate_population(num_individuals, distance_matrix, vehicle_capacity, num_vehicles):
    return [generate_individual(distance_matrix, vehicle_capacity, num_vehicles) for _ in range(num_individuals)]


In [25]:

def rank_population_by_fitness(population, distance_matrix):
    fitness_scores = [total_distance_of_routes(individual, distance_matrix) for individual in population]
    ranked_population = sorted(enumerate(fitness_scores), key=lambda x: x[1])
    return ranked_population

def roulette_wheel_selection(ranked_population, elitism_size):
    # Extract the best individuals based on elitism size
    elites = ranked_population[:elitism_size]
    
    # Extracting the fitness scores
    fitness_scores = [score for _, score in ranked_population]

    # Normalize the fitness scores so that they sum to 1
    total_fitness = sum(fitness_scores)
    normalized_fitness_scores = [score / total_fitness for score in fitness_scores]

    # Calculate the cumulative sum of the normalized fitness scores
    cumulative_sums = np.cumsum(normalized_fitness_scores)

    # Select individuals using the cumulative sums
    selected_individuals = elites.copy()
    for _ in range(len(ranked_population) - elitism_size):
        random_number = np.random.random()
        for i, cumulative_sum in enumerate(cumulative_sums):
            if random_number <= cumulative_sum:
                selected_individuals.append(ranked_population[i])
                break

    return selected_individuals




In [26]:
def create_mating_pool(selected_individuals, population):
    mating_pool = [population[individual_idx] for individual_idx, _ in selected_individuals]
    return mating_pool

def ordered_crossover(parent1, parent2, distance_matrix, vehicle_capacity):
    # Determine crossover points
    start_pos = random.randint(0, len(parent1) - 1)
    end_pos = random.randint(start_pos + 1, len(parent1))

    # Initialize offspring with the subset from parent1
    offspring = [[] for _ in range(len(parent1))]
    offspring[start_pos:end_pos] = [list(route) for route in parent1[start_pos:end_pos]]

    # Customers that are already in the offspring
    included_customers = {c for route in offspring[start_pos:end_pos] for c in route}

    # Fill the remainder of the routes with parent2, considering capacity constraints
    for i, route in enumerate(parent2):
        route_demand = 0
        for c in route:
            if c not in included_customers and route_demand + customer_demands[c] <= vehicle_capacity:
                offspring[i].append(c)
                included_customers.add(c)
                route_demand += customer_demands[c]

    # Ensure all customers are included by filling empty spots
    for i, route in enumerate(parent1 + parent2):
        route_demand = sum(customer_demands[c] for c in offspring[i % len(offspring)])
        for c in route:
            if c not in included_customers and route_demand + customer_demands[c] <= vehicle_capacity:
                offspring[i % len(offspring)].append(c)
                included_customers.add(c)
                route_demand += customer_demands[c]

    return offspring

def breed_population(mating_pool, elitism_size, vehicle_capacity):
    # Retaining the best individuals (elitism)
    elites = mating_pool[:elitism_size]

    # Using crossover to fill the rest of the new population
    offspring = []
    for i in range(len(mating_pool) - elitism_size):
        parent1 = mating_pool[i + elitism_size]
        parent2 = mating_pool[(i + elitism_size + 1) % len(mating_pool)] # Looping back around the mating pool
        child = ordered_crossover(parent1, parent2, vehicle_capacity)
        
        # Only add child to offspring if there are no empty routes
        if all(route for route in child):
            offspring.append(child)

    # If we discarded some children and have fewer offspring than required, 
    # fill up the rest from the top individuals of the previous generation.
    deficit = len(mating_pool) - elitism_size - len(offspring)
    if deficit > 0:
        offspring.extend(mating_pool[elitism_size:elitism_size+deficit])

    # Combine elites and offspring to form the new population
    children = elites + offspring
    return children


In [27]:
import random

def swap_mutation(child, mutation_rate):
    if random.random() < mutation_rate:
        # Select a random route with at least two cities
        valid_routes = [route for route in child if len(route) >= 2]
        if not valid_routes:
            return child  # No valid routes to mutate

        # Select a random route from valid routes
        route_idx = random.randint(0, len(valid_routes) - 1)
        route = valid_routes[route_idx]

        # Select two random indices within the chosen route
        idx1, idx2 = random.sample(range(len(route)), 2)

        # Swap the cities at these two positions
        route[idx1], route[idx2] = route[idx2], route[idx1]
        child[child.index(valid_routes[route_idx])] = route  # Update the child with the mutated route

    return child

def mutate_population(population, mutation_rate):
    mutated_population = []

    for individual in population:
        mutated_individual = swap_mutation(individual, mutation_rate)
        mutated_population.append(mutated_individual)

    # Remove empty routes from mutated population
    for individual in mutated_population:
        individual[:] = [route for route in individual if route]

    return mutated_population


In [28]:
def nextGeneration(currentGen, elitism_size, mutation_rate, distance_matrix):
    popRanked = rank_population_by_fitness(currentGen, distance_matrix)
    selectionResults = roulette_wheel_selection(popRanked, elitism_size)
    matingpool = create_mating_pool(selectionResults, currentGen)

    children = breed_population(matingpool, elitism_size, vehicle_capacity, distance_matrix)
    nextGeneration = mutate_population(children, mutation_rate)
    return nextGeneration

def geneticAlgorithm(distance_matrix, num_individuals, vehicle_capacity, num_vehicles, elitism_size, mutation_rate, generations):
    pop = generate_population(num_individuals, distance_matrix, vehicle_capacity, num_vehicles)

    for i in range(0, generations):
        pop = nextGeneration(pop, elitism_size, mutation_rate, distance_matrix)

    bestRouteIndex = rank_population_by_fitness(pop, distance_matrix)[0][0]
    bestRoute = pop[bestRouteIndex]
    return bestRoute


In [ ]:
import time
num_individuals = 50
vehicle_capacity = 100
elitism_size = 1
mutation_rate = 0.01
generations = 10
num_vehicles = 9


import time

def multiple_runs(runs=5):
    results = []
    
    for run in range(runs):
        start_time = time.process_time()  # Start measuring CPU time
        
        best_route = geneticAlgorithm(customer_data, num_individuals, vehicle_capacity, num_vehicles, elitism_size, mutation_rate, generations)
        
        end_time = time.process_time()  # End measuring CPU time
        
        run_time = end_time - start_time
        
        results.append({
            'run_number': run+1,
            'best_route': best_route,
            'best_distance': total_distance_of_routes(best_route, customer_data),
            'run_time': run_time
        })
        
        print(f"Run {run+1} completed in {run_time:.2f} seconds.")
    
    return results

results = multiple_runs()

# If you want to see the results
for result in results:
    print(f"Run {result['run_number']}: Best Distance = {result['best_distance']}, Time = {result['run_time']:.2f} seconds")



In [20]:
filename = "Vehicle_Routing_dataset.csv"
df = pd.read_csv(filename, delimiter=';',usecols=['Node', 'Demand', 'x-coordinate', 'y-coordinate'])

customer_data = {index: (demand, (x, y)) for index, demand, x, y in df.itertuples(index=False)}


print(customer_data)





{0: (0, (36, 64)), 1: (3, (94, 47)), 2: (12, (10, 23)), 3: (25, (16, 46)), 4: (4, (25, 79)), 5: (11, (41, 30)), 6: (20, (81, 45)), 7: (21, (14, 79)), 8: (10, (42, 56)), 9: (20, (90, 17)), 10: (13, (41, 39)), 11: (14, (21, 14)), 12: (16, (41, 46)), 13: (17, (65, 96)), 14: (11, (13, 49)), 15: (36, (21, 14)), 16: (6, (57, 2)), 17: (7, (14, 42)), 18: (21, (66, 62)), 19: (11, (58, 96)), 20: (17, (5, 51)), 21: (22, (41, 50)), 22: (10, (50, 99)), 23: (19, (84, 85)), 24: (21, (97, 90)), 25: (23, (47, 76)), 26: (19, (11, 54)), 27: (15, (60, 97)), 28: (22, (60, 89)), 29: (7, (58, 68)), 30: (11, (30, 93)), 31: (15, (9, 60)), 32: (22, (47, 44)), 33: (12, (19, 40)), 34: (24, (15, 40)), 35: (25, (88, 21)), 36: (2, (33, 58)), 37: (15, (21, 51)), 38: (18, (57, 7)), 39: (13, (81, 6)), 40: (3, (49, 6)), 41: (20, (51, 78)), 42: (14, (9, 62)), 43: (10, (84, 36)), 44: (10, (95, 76)), 45: (66, (89, 44)), 46: (10, (10, 49)), 47: (7, (69, 16)), 48: (12, (75, 66)), 49: (24, (97, 11)), 50: (5, (74, 69)), 51: (1

In [5]:
import numpy as np

def distance_between_customers(c1, c2):
    """Calculate Euclidean distance between two customers."""
    return np.linalg.norm(np.array(c1[1:]) - np.array(c2[1:]))

def route_distance(route):
    """Calculate the total distance of a route."""
    return sum(distance_between_customers(customer_data[route[i]], customer_data[route[i + 1]]) for i in range(len(route) - 1))




In [ ]:
updated_solution = [[0, 4, 7, 42, 31, 20, 46, 26, 0], [0, 36, 11, 15, 51, 2, 17, 14, 0], [0, 37, 3, 34, 33, 21, 0], [0, 1, 45, 6, 8, 0], [0, 25, 41, 29, 0], [0, 23, 52, 24, 44, 50, 48, 18, 0], [0, 32, 38, 16, 40, 53, 5, 10, 12, 0], [0, 30, 22, 19, 27, 13, 54, 28, 0], [0, 47, 39, 49, 9, 35, 43, 0]]

distances = [route_distance(route) for route in updated_solution]
print(total_distance_distance(route))
# Print the distances of each route in updated_solution
#print(sum(distances))

In [69]:
print(customers)

54


In [289]:
import random


def generate_population(num_routes, customers_data, vehicle_capacity):
    customers = list(range(1,len(customer_data)))
    random.shuffle(customers)
    population = []
    while len(population) < num_routes:
        route = [0]
        route_demand = 0
        for c in customers:
            if route_demand + c <= vehicle_capacity:
                route.append(c)
                route_demand += c
        population.append(route)
        customers = [c for c in customers if c not in route]
        route.append(0)
    return population


num_vehicles = 9
vehicle_capacity = 100
current_population = generate_population(num_vehicles, customers, vehicle_capacity)
for i, route in enumerate(current_population):
    print(f"Route #{i+1}: {route}")





Route #1: [0, 54, 7, 31, 1, 5, 2, 0]
Route #2: [0, 34, 10, 36, 18, 0]
Route #3: [0, 50, 23, 20, 4, 3, 0]
Route #4: [0, 41, 37, 17, 0]
Route #5: [0, 45, 29, 11, 12, 0]
Route #6: [0, 47, 39, 9, 0]
Route #7: [0, 44, 16, 27, 8, 0]
Route #8: [0, 46, 35, 15, 0]
Route #9: [0, 19, 42, 33, 6, 0]


In [290]:
distances = [route_distance(route) for route in current_population]

print(distances)

print(sum(distances))

[350.2182525489161, 141.83332226274277, 238.72312372985692, 103.39360929310378, 217.73000533760788, 159.67173858858976, 293.219093238415, 232.4562711769598, 233.68764295769333]
1970.933059133885


In [293]:
def roulette_wheel_selection(population, distances, num_parents):
    probabilities = [fit / sum(distances) for fit in distances]
    parents = random.choices(population, probabilities, k=2)
    return parents

num_parents = 2
parent1, parent2 = roulette_wheel_selection(current_population, distances, num_parents)

print(parent1, parent2)
print(route_distance(parent1))
print(route_distance(parent2))

[0, 50, 23, 20, 4, 3, 0] [0, 54, 7, 31, 1, 5, 2, 0]
238.72312372985692
350.2182525489161


In [292]:
def crossover(parent1, parent2, vehicle_capacity):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    # Repair mechanism - Route Partitioning for Child 1
    if sum(child1[1:-1]) > vehicle_capacity:
        for i in range(1, len(child1) - 1):
            if sum(child1[1:i]) <= vehicle_capacity:
                child2 = child1[i:] + child2
                child1 = child1[:i]
                break

    # Repair mechanism - Route Partitioning for Child 2
    if sum(child2[1:-1]) > vehicle_capacity:
        for i in range(1, len(child2) - 1):
            if sum(child2[1:i]) <= vehicle_capacity:
                child1 = child2[i:] + child1
                child2 = child2[:i]
                break

    return child1, child2

crossover(parent1, parent2, vehicle_capacity)
print(child1,child2)
print(route_distance(child1))
print(route_distance(child2))

[0, 17, 27, 14, 25, 13, 0] [0, 22, 38, 39, 0]
283.50181172716583
227.3927065933167


In [281]:
def mutate(route, mutation_probability):
    """Swap two customers in a route."""
    if random.random() < mutation_probability:
        idx1, idx2 = random.sample(range(1, len(population) - 1), 2)
        population[idx1], population[idx2] = population[idx2], population[idx1]
    return population

In [282]:
mutation_probability = 0.2
mutate_population = mutate(population, mutation_probability)
print(mutate_population)

[[0, 45, 33, 9, 3, 6, 1, 2, 0], [0, 44, 47, 4, 5, 0], [0, 36, 29, 20, 15, 0], [0, 50, 18, 32, 0], [0, 51, 25, 11, 10, 0], [0, 53, 43, 0], [0, 39, 42, 17, 0], [0, 54, 37, 8, 0], [0, 26, 49, 7, 14, 0]]


In [ ]:
def create_next_generation(population, num_parents, mutation_probability, vehicle_capacity):

In [283]:
def create_next_generation(population, num_parents, mutation_probability, vehicle_capacity):
    distances = total_distance_distance(route)
    new_generation = []

#     # Elitism: select top 10% individuals
#     elite_offset = int(0.1 * len(population))
#     elites = sorted(range(len(distances)), key=lambda x: distances[x])[:elite_offset]
#     for idx in elites:
#         new_generation.append(population[idx])
        
    parents = roulette_wheel_selection(population, distances, num_parents)
    for _ in range(len(population)):
        children = crossover(parent1, parent2)
        for route in children:
            mutate(route, mutation_probability)
        new_generation.append(children)    

    return new_generation

In [219]:
def single_route_distances(population):
    return [route_distance(route) for route in population]

def total_distances(distances):
    return sum(single_route_distances(population))
    

In [256]:
def create_next_generation(population, num_parents, mutation_probability, vehicle_capacity):
    new_generation = []

    for _ in range(len(population)):
        parents = roulette_wheel_selection(population, [route_distance(route) for route in population], num_parents)
        print('parents',parents)
        child1, child2 = single_point_crossover(parent[0], parent[1])
        new_generation.append(child1)
        new_generation.append(child2)
        print('crossover',new_generation)
    
    # Apply mutation to the new generation
    for route in new_generation:
        mutate(route, mutation_probability)

    return new_generation


In [ ]:
num_parents = 2
num_mutations = 0.2
new_population = create_next_generation(current_population, distances, num_parents, num_mutations)
print(new_population)
print(len(new_population))


In [210]:
import time

# Define the number of runs
num_runs = 30

# Define the vehicle capacity and number of vehicles
vehicle_capacity = 100
num_vehicles = 9

# Initialize the best solution and its total distance
best_solution = None
best_distance = float('inf')

# Start the timer
start_time = time.time()

for run in range(num_runs):
    # Generate the initial population
    current_population = generate_population(num_vehicles, customers, vehicle_capacity)
    
    # Run the optimization for a specific number of generations (you can adjust this)
    num_generations = 100
    for generation in range(num_generations):
        current_distances = sum([route_distance(route) for route in current_population])
        current_population = create_next_generation(current_population, num_parents=2, mutation_probability=0.2, vehicle_capacity=vehicle_capacity)
    
    # Calculate the total distance of the best solution in the current run
    current_best_distance = min([route_distance(route) for route in current_population])

    # Check if the current solution is better than the best solution found so far
    if current_best_distance < best_distance:
        best_solution = current_population
        best_distance = current_best_distance

# End the timer
end_time = time.time()

# Calculate the total CPU time
cpu_time = end_time - start_time

# Print the best solution and its total distance
print("Best Solution:")
for i, route in enumerate(best_solution):
    print(f"Route #{i+1}: {route}")
print("Total Distance:", best_distance)

# Print the total CPU time
print("Total CPU Time:", cpu_time, "seconds")


total_distance: 2300.6784423348995
total_distance: 2115.671023704934
total_distance: 1984.9204500490277
total_distance: 1826.460779633685
total_distance: 1853.3854814969802
total_distance: 1785.3626260561812
total_distance: 1522.5225582443538
total_distance: 1325.0514440203679
total_distance: 1185.7849211063808
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1624842239764
total_distance: 1090.1

total_distance: 565.1898850936656
total_distance: 2195.0526152049683
total_distance: 1406.6804477910057
total_distance: 1365.2511526508551
total_distance: 1294.5072045801342
total_distance: 1259.6643016845292
total_distance: 1176.07832270518
total_distance: 1208.633770689256
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.6291294105783
total_distance: 1122.629

total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance: 782.6840356718232
total_distance

total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance: 792.1998563077593
total_distance

total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance: 744.6432262673763
total_distance

...

total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 783.9779016681906
total_distance: 2052.8649534075444
total_distance: 1661.701088865482
total_distance: 1660.0499639493253
total_distance: 1482.7940785226597
total_distance: 1425.0897630201894
total_distance: 1289.1247984631575
total_distance: 1220.4969704664938
total_distance: 1116.307109498644
total_distance: 1040.8651079039967
total_distance: 995.8115709775323
total_distance: 928.2312655878357
total_distance: 928.2312655878357
total_d

total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_distance: 653.806152132146
total_dist

total_distance: 660.3883312719375
total_distance: 660.3883312719375
total_distance: 660.3883312719375
total_distance: 660.3883312719375
total_distance: 660.3883312719375
total_distance: 2088.2174306090446
total_distance: 1564.5377175913645
total_distance: 1417.5153612561887
total_distance: 1376.2983823576533
total_distance: 1279.5128009771067
total_distance: 1283.8781392897583
total_distance: 1168.801734079948
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.9848473807624
total_distance: 1095.98484

total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251.3066970512243
total_distance: 1251

total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance: 822.3584963617955
total_distance

In [25]:
import random

def generate_random_population(customers, vehicle_capacity):
    routes = []
    remaining_customers = customers.copy()
    while remaining_customers:
        route = [0]
        route_demand = 0
        while remaining_customers and route_demand <= vehicle_capacity:
            next_customer = random.choice(remaining_customers)
            new_demand = route_demand + customer_data[next_customer][0]
            if new_demand <= vehicle_capacity:
                route.append(next_customer)
                route_demand = new_demand
                remaining_customers.remove(next_customer)
        route.append(0)
        routes.append(route)
    return routes


In [26]:
def roulette_wheel_selection(population, fitness, num_parents):
    """Select parents using a probability proportionate to their fitness."""
    probabilities = [fit / sum(fitness) for fit in fitness]
    return random.choices(population, weights=probabilities, k=num_parents)

def crossover(parent1, parent2):
    """Perform two-point crossover."""
    start = random.randint(1, len(parent1) - 2)
    end = random.randint(start, len(parent1) - 1)
    child = [-1] * len(parent1)
    child[start:end] = parent1[start:end]
    for i in parent2:
        if i not in child:
            for j, val in enumerate(child):
                if val == -1:
                    child[j] = i
                    break
    return child

def mutate(route, mutation_probability):
    """Swap two customers in a route."""
    if random.random() < mutation_probability:
        idx1, idx2 = random.sample(range(1, len(route) - 1), 2)
        route[idx1], route[idx2] = route[idx2], route[idx1]
    return route


In [27]:
def create_next_generation(population, num_parents, mutation_probability):
    fitness = [sum(route_distance(route) for route in individual) for individual in population]
    new_generation = []
    
    # Elitism: select top 10% individuals
    elite_offset = int(0.1 * len(population))
    elites = sorted(range(len(fitness)), key=lambda x: fitness[x])[:elite_offset]
    for idx in elites:
        new_generation.append(population[idx])

    # Create the rest of the population
    for _ in range(len(population) - elite_offset):
        parents = roulette_wheel_selection(population, fitness, num_parents)
        child = crossover(parents[0], parents[1])
        for route in child:
            mutate(route, mutation_probability)
        new_generation.append(child)
    
    return new_generation


In [ ]:
# Genetic Algorithm parameters
population_size = 50
num_parents = 2
mutation_probability = 0.2
num_generations = 100

# Initial population
population = [generate_random_population(list(range(1, len(customer_data))), vehicle_capacity) for _ in range(population_size)]

best_solution = None
best_fitness = float('inf')

for generation in range(num_generations):
    fitness = [sum(route_distance(route) for route in individual) for individual in population]
    
    if min(fitness) < best_fitness:
        best_fitness = min(fitness)
        best_solution = population[fitness.index(min(fitness))]

    population = create_next_generation(population, num_parents, mutation_probability)

# Display results
print("Best Solution:")
for i, route in enumerate(best_solution):
    print(f"Route #{i+1}: {route}")
print(f"Total Travelling Cost: {best_fitness}")


In [ ]:
import random
import numpy as np

# Define the number of vehicles and the vehicle capacity
num_vehicles = 9
vehicle_capacity = 100
num_routes = 9
customers = 54


def distance_between_customers(c1, c2):
    # Calculate Euclidean distance between two customers using their x and y coordinates
    return np.linalg.norm(np.array(c1[1:]) - np.array(c2[1:]))

def route_distance(route):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance_between_customers(customer_data[route[i]], customer_data[route[i + 1]])
    return total_distance

def generate_random_population(customers, vehicle_capacity):
    routes = []
    remaining_customers = customers.copy()
    while remaining_customers:
        route = [0]
        route_demand = 0
        while remaining_customers and route_demand <= vehicle_capacity:
            next_customer = random.choice(remaining_customers)
            new_demand = route_demand + customer_data[next_customer][0]
            if new_demand <= vehicle_capacity:
                route.append(next_customer)
                route_demand = new_demand
                remaining_customers.remove(next_customer)
        route.append(0)
        routes.append(route)
    return routes


def evaluate_population(population):
    fitness = [route_distance(route) for route in population]
    total_fitness = sum(fitness)
    return fitness, total_fitness

def roulette_wheel_selection(population, fitness, num_parents):
    probabilities = [fit / sum(fitness) for fit in fitness]
    parents = random.choices(population, probabilities, k=num_parents)
    return parents

def crossover(parent1, parent2):
    start = random.randint(1, len(parent1) - 2)
    end = random.randint(start, len(parent1) - 1)
    child = [-1] * len(parent1)
    child[start:end] = parent1[start:end]
    for i in range(len(parent2)):
        if parent2[i] not in child:
            for j in range(len(child)):
                if child[j] == -1:
                    child[j] = parent2[i]
                    break
    return child

def mutate(route, mutation_probability):
    if random.random() < mutation_probability:
        idx1, idx2 = random.sample(range(1, len(route) - 1), 2)
        route[idx1], route[idx2] = route[idx2], route[idx1]
    return route

def create_next_generation(population, num_parents, mutation_probability):
    fitness, _ = evaluate_population(population)
    new_generation = []
    elite_offset = int(0.1 * len(population))
    new_generation.extend(population[:elite_offset])
    for _ in range(len(population) - elite_offset):
        parents = roulette_wheel_selection(population, fitness, num_parents)
        child = crossover(parents[0], parents[1])
        child = mutate(child, mutation_probability)
        new_generation.append(child)
    return new_generation

# Genetic Algorithm parameters
population_size = 50
num_parents = 2
mutation_probability = 0.2
num_generations = 100

# Initialize the initial population
population = [generate_random_population(list(range(1, customers + 1)), vehicle_capacity) for _ in range(population_size)]

# Evolution loop
best_solution = None
best_fitness = float('inf')
for generation in range(num_generations):
    fitness, _ = evaluate_population(population)
    best_solution_in_generation = population[fitness.index(min(fitness))]
    if min(fitness) < best_fitness:
        best_solution = best_solution_in_generation.copy()
        best_fitness = min(fitness)
    population = create_next_generation(population, num_parents, mutation_probability)

# Print the best solution found
print("Best Solution:")
for i, route in enumerate(best_solution):
    print(f"Route #{i+1}: {route}")

print(f"Total Travelling Cost: {best_fitness}")


In [188]:


# Define the number of vehicles and the vehicle capacity
num_vehicles = 9
vehicle_capacity = 100
num_routes = 9
customers = 54

# # Define the number of customers and create a list of customer indices
# customer_indices = list(range(1, customers + 1))





In [189]:
# import random

# def generate_population(num_routes, customers, vehicle_capacity):
#     random.shuffle(customers)
#     population = []
#     while len(population) < num_routes:
#         route = [0]
#         route_demand = 0
#         for c in customers:
#             if route_demand + c <= vehicle_capacity:
#                 route.append(c)
#                 route_demand += c
#         population.append(route)
#         customers = [c for c in customers if c not in route]
#         route.append(0)
#     return population

# customers = list(range(1, 55))
# vehicle_capacity = 100
# population = generate_population(9, customers, vehicle_capacity)
# for i, route in enumerate(population):
#     print(f"Route #{i+1}: {route}")


Route #1: [0, 40, 18, 1, 3, 12, 5, 9, 7, 4, 0]
Route #2: [0, 43, 35, 15, 6, 0]
Route #3: [0, 36, 44, 11, 2, 0]
Route #4: [0, 26, 29, 33, 10, 0]
Route #5: [0, 38, 21, 25, 13, 0]
Route #6: [0, 28, 22, 16, 34, 0]
Route #7: [0, 20, 47, 27, 0]
Route #8: [0, 46, 32, 19, 0]
Route #9: [0, 54, 37, 8, 0]


In [191]:
import numpy as np


def evaluate_population(population, customer_data):
    fitness = []
    for route in population:
        route_distance = 0
        for i in range(len(route) - 1):
            start = np.array(customer_data[route[i]][1], dtype=float)
            end = np.array(customer_data[route[i + 1]][1], dtype=float)
            route_distance += np.linalg.norm(start - end)
        fitness.append(route_distance)
    return fitness,sum(fitness)


In [192]:
num_parents = 2


def roulette_wheel_selection(populations, fitness, num_parents):
    fitness_sum = sum(fitness)
    probability = [fit / fitness_sum for fit in fitness]
    parents = []
    for i in range(num_parents):
        random_value = random.uniform(0, 1)
        for j, p in enumerate(probability):
            random_value -= p
            if random_value <= 0:
                parents.append(populations[j])
                break
    return parents



In [196]:
populations = []
total_travelling_costs = []
num_populations = 50
for i in range(num_populations):
    populations.append(generate_population(num_routes, customers, vehicle_capacity))

for i in populations:
    total_travelling_costs.append(evaluate_population(i, customer_data))



In [ ]:
parents = roulette_wheel_selection(population, total_travelling_costs, num_parents)
parent1 = parents[0]
parent2 = parents[1]
def crossover(parent1, parent2):
    start = random.randint(0, len(parent1))
    end = random.randint(start, len(parent1))
    child = [-1] * len(parent1)
    child[start:end] = parent1[start:end]
    for i in range(len(parent2)):
        if parent2[i] not in child:
            for j in range(len(child)):
                if child[j] == -1:
                    child[j] = parent2[i]
                    break
    return child

mutation_probability = 0.1
def mutate(populations, mutation_probability):
    for i in range(len(populations)):
        if random.random() < mutation_probability:
            swap_with = random.randint(0, len(route) - 1)
            populations[i], populations[swap_with] = populations[swap_with], populations[i]
    return populations



elitism = 0.1
current_generation = populations
def create_next_generation(current_generation, fitness, mutation_probability):
    new_generation = []
    elite_offset = int(elitism * len(current_generation))
    new_generation.extend(current_generation[:elite_offset])
    total_fitness = sum(fitness)
    for i in range(elite_offset, len(current_generation)):
        parent1 = roulette_wheel_selection(current_generation, fitness, num_parents)[0]
        parent2 = roulette_wheel_selection(current_generation, fitness, num_parents)[1]
        child = crossover(parent1, parent2)
        child = mutate(child, mutation_probability)
        new_generation.append(child)
    return new_generation

In [ ]:
max_generations = 100

for generation in range(max_generations):
    # Evaluate the fitness of the current population
    fitness,_ = evaluate_population(current_generation, customer_data)
    # Select parents using roulette wheel selection
    parents = roulette_wheel_selection(current_generation, fitness, num_parents)
    # Create a new population by performing crossover and mutation
    current_generation = create_next_generation(current_generation, fitness, mutation_probability)

# The final population after the loop is the solution
best_route = current_generation[0]
best_fitness = fitness[0]

num_generations = 100
best_solution = None
best_fitness = float('inf')

for generation in range(num_generations):
    fitness, fitness_sum = evaluate_population(current_generation, customer_data)
    best_solution_in_generation = current_generation[fitness.index(min(fitness))]
    if min(fitness) < best_fitness:
        best_solution = best_solution_in_generation
        best_fitness = min(fitness)
    current_generation = create_next_generation(current_generation, fitness, mutation_probability)



In [187]:
b = create_next_generation(current_generation, fitness, mutation_probability)

b_cost = []
for i in b:
  b_cost.append(evaluate_population(i, customer_data))
print(min(b_cost))

1335.0344058126325
